# Save sample data for testing

In [1]:
import boto3
import os
from tqdm import tqdm

s3 = boto3.client("s3")

# List all objects helper internal function
def list_all_objects(bucket, prefix):
    # Create a paginator for list_objects_v2
    paginator = s3.get_paginator('list_objects_v2')

    # Use the paginator to iterate through all pages
    all_objects = []
    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        if 'Contents' in page:
            all_objects.extend(page['Contents'])

    return all_objects

bucket = 'udacity-deeplearning-project'
prefix = 'data'
sample_prefix = 'sample'

all_files = list_all_objects(bucket, prefix)

sample_files = [file_meta for file_meta in all_files if '001.Affenpinscher' in file_meta.get("Key")]

for file_meta in tqdm(sample_files):
    key = file_meta.get("Key")
    
    # Move the data from s3 to a different prefix
    dirname = os.path.dirname(key)
    if dirname:
        os.makedirs(dirname, exist_ok=True)

    s3.download_file(bucket, key, key)
    # s3.copy_object(
    #     Bucket=bucket,
    #     CopySource=f"{bucket}/{key}",
    #     Key=f"{sample_prefix}/{key}"
    # )

100%|██████████| 80/80 [00:06<00:00, 11.57it/s]


## Testing Estimator Locally Prior to Deployment to ECR

In [13]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
import sagemaker
import subprocess

subprocess.run(['sh', 'docker-build.sh'])

role=get_execution_role()

bucket = 'udacity-deeplearning-project'
sample_prefix = 'sample'

hyperparameters = {
    'num_classes': 133,
    'epochs': 1
}

bucket = 'udacity-deeplearning-project'
s3_output_location = f"s3://{bucket}/outputs"

estimator=Estimator(
    image_uri='udacity-sagemaker-hpo',
    role=role,
    instance_count=1,
    instance_type='local',
    output_path=s3_output_location,
    hyperparameters=hyperparameters
)

model_inputs = {
    "train": "file://./data/test",
    "test": "file://./data/valid"
}

estimator.fit(inputs=model_inputs, logs=True ,wait=True)

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sha256:1d70e334e671237f62dbf79d0774e9f155ead75f6f12b5399e469bbe48c7ed1d
REPOSITORY                                                           TAG       IMAGE ID       CREATED                  SIZE
udacity-sagemaker-hpo                                                latest    1d70e334e671   Less than a second ago   3.92GB
598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo   latest    b0d4eacb43b3   10 minutes ago           3.92GB
598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo   <none>    fd1260f0a822   32 minutes ago           3.92GB
598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo   <none>    64baee4f0962   2 hours ago              3.92GB


INFO:sagemaker:Creating training-job with name: udacity-sagemaker-hpo-2024-11-18-17-22-23-703
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' is not installed. Proceeding to check for 'docker-compose' CLI.
INFO:sagemaker.local.image:'Docker Compose' found using Docker Compose CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemaker.local.i

 Container 5zqoh5u1rp-algo-1-wympb  Creating
 Container 5zqoh5u1rp-algo-1-wympb  Created
Attaching to 5zqoh5u1rp-algo-1-wympb
5zqoh5u1rp-algo-1-wympb  | sed: can't read changehostname.c: No such file or directory
5zqoh5u1rp-algo-1-wympb  | gcc: error: changehostname.c: No such file or directory
5zqoh5u1rp-algo-1-wympb  | gcc: fatal error: no input files
5zqoh5u1rp-algo-1-wympb  | compilation terminated.
5zqoh5u1rp-algo-1-wympb  | gcc: error: changehostname.o: No such file or directory
5zqoh5u1rp-algo-1-wympb  | ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
5zqoh5u1rp-algo-1-wympb  | 2024-11-18 17:22:24,713 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
5zqoh5u1rp-algo-1-wympb  | 2024-11-18 17:22:24,716 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
5zqoh5u1rp-algo-1-wympb  | 2024-11-18 17:22:24,729 sagemaker_pytorch_container.tra

INFO:sagemaker.local.image:===== Job Complete =====


5zqoh5u1rp-algo-1-wympb exited with code 0
Aborting on container exit...
 Container 5zqoh5u1rp-algo-1-wympb  Stopping
 Container 5zqoh5u1rp-algo-1-wympb  Stopped


## Testing Deployed Estimator

In [14]:
!sh push-container.sh

push-container.sh: line 1: fg: no job control
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.0s (1/1) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 2B                                         0.0s
ERROR: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory
The push refers to repository [598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo]

bf18a086: Preparing 
61449057: Preparing 
95045e04: Preparing 
e9708ca1: Preparing 
8f6060c6: Preparing 
a3c12226: Preparing 
62daa95e: Preparing 
8fe1cb59: Preparing 
061a5b0d: Preparing 
3ff1bf08:

In [15]:
model_inputs = {
    "train": sagemaker.inputs.TrainingInput(
        s3_data=f"s3://{bucket}/{sample_prefix}/data/train/",
        content_type="application/x-image"
    ),
    "test": sagemaker.inputs.TrainingInput(
        s3_data=f"s3://{bucket}/{sample_prefix}/data/valid/",
        content_type="application/x-image"
    )
}

In [16]:
estimator=Estimator(
    image_uri='598308907998.dkr.ecr.us-east-1.amazonaws.com/udacity-sagemaker-hpo',
    role=role,
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    hyperparameters=hyperparameters
)

estimator.fit(inputs=model_inputs, logs=True)

INFO:sagemaker:Creating training-job with name: udacity-sagemaker-hpo-2024-11-18-17-22-41-489


2024-11-18 17:22:42 Starting - Starting the training job
2024-11-18 17:22:42 Pending - Training job waiting for capacity......
2024-11-18 17:23:40 Pending - Preparing the instances for training......
2024-11-18 17:24:47 Downloading - Downloading the training image......
2024-11-18 17:25:39 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
2024-11-18 17:25:51,187 sagemaker-training-toolkit 